# Define fxns

In [ ]:
import torch
from torch.utils.benchmark import Timer
from torchvision.io.image import decode_jpeg, read_file, read_image
from torchvision import transforms as T

# pil
from PIL import Image

# cv2
import cv2

# skimage
from skimage import io

# rasterio 
import rasterio 

import inspect
import glob
import numpy as np
import pandas as pd
import tifffile as tiff
import time


In [9]:
convert_to_tensor = T.transforms.ToTensor()

def load_image_raw(image_path):
    """
    Loads an image from the specified file path and returns the binary data.

    Args:
        image_path (str): The path to the image file.

    Returns:
        bytes: The binary data of the image.
    """
    f = open(image_path, 'rb')
    bytes = f.read()
    f.close()
    return bytes

def pil_to_numpy(im):
    """
    Converts a PIL Image object to a NumPy array.
    Source : Fast import of Pillow images to NumPy / OpenCV arrays Written by Alex Karpinsky

    Args:
        im (PIL.Image.Image): The input PIL Image object.

    Returns:
        numpy.ndarray: The NumPy array representing the image.
    """
    im.load()

    # Unpack data
    e = Image._getencoder(im.mode, "raw", im.mode)
    e.setimage(im.im)

    # NumPy buffer for the result
    shape, typestr = Image._conv_type_shape(im)
    data = np.empty(shape, dtype=np.dtype(typestr))
    mem = data.data.cast("B", (data.data.nbytes,))

    bufsize, s, offset = 65536, 0, 0
    while not s:
        l, s, d = e.encode(bufsize)

        mem[offset:offset + len(d)] = d
        offset += len(d)
    if s < 0:
        raise RuntimeError("encoder error %d in tobytes" % s)
    return data



# Image loading functions : 
def load_image_rasterio(image_path):
    # Open the image using rasterio and read its contents into an array
    with rasterio.open(image_path, 'r') as dataset:
        array = dataset.read()
    
    # Convert the array to a PyTorch tensor
    tensor = torch.from_numpy(array)
    
    # Return the tensor
    return tensor

def load_image_nvjpegl_cpu(image_path):
    """
    Loads an image from the specified file path using NVJPEG decoder on CPU and returns a PyTorch tensor.

    Args:
        image_path (str): The path to the image file.

    Returns:
        torch.Tensor: The PyTorch tensor representing the image.
    """
    data = read_file(image_path)
    tensor = decode_jpeg(data, device="cpu")
    return tensor

def load_image_nvjpegl_gpu(image_path):
    """
    Loads an image from the specified file path using NVJPEG decoder on GPU and returns a PyTorch tensor.

    Args:
        image_path (str): The path to the image file.

    Returns:
        torch.Tensor: The PyTorch tensor representing the image.
    """
    data = read_file(image_path)
    tensor = decode_jpeg(data, device="cuda:0")
    return tensor

def load_image_pil_accelerated(image_path):
    """
    Loads an image from the specified file path using PIL and returns a PyTorch tensor.

    Args:
        image_path (str): The path to the image file.

    Returns:
        torch.Tensor: The PyTorch tensor representing the image.
    """
    image = Image.open(image_path)
    array = pil_to_numpy(image)
    tensor = torch.from_numpy(array)
    return tensor


def load_image_skimage(image_path):
    """
    Loads an image from the specified file path using scikit-image and returns a PyTorch tensor.

    Args:
        image_path (str): The path to the image file.

    Returns:
        torch.Tensor: The PyTorch tensor representing the image.
    """
    image = io.imread(image_path)
    tensor = torch.from_numpy(image)
    return tensor

def load_image_pil(image_path):
    """
    Loads an image from the specified file path using PIL and returns a PyTorch tensor.

    Args:
        image_path (str): The path to the image file.

    Returns:
        torch.Tensor: The PyTorch tensor representing the image.
    """
    image = Image.open(image_path)
    array = np.array(image)
    tensor = torch.from_numpy(array)
    return tensor

def load_image_cv2(image_path):
    """
    Loads an image from the specified file path using OpenCV (cv2) and returns a PyTorch tensor.

    Args:
        image_path (str): The path to the image file.

    Returns:
        torch.Tensor: The PyTorch tensor representing the image.
    """
    image = cv2.imread(image_path)
    array = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    tensor = torch.from_numpy(array)
    return tensor

def load_image_pil_to_tensor(image_path):
    """
    Loads an image from the specified file path using PIL and returns a PyTorch tensor.

    Args:
        image_path (str): The path to the image file.

    Returns:
        torch.Tensor: The PyTorch tensor representing the image.
    """
    image = Image.open(image_path)
    tensor = convert_to_tensor(image)
    return tensor

def load_image_cv_to_tensor(image_path):
    """
    Loads an image from the specified file path using OpenCV (cv2) and returns a PyTorch tensor.

    Args:
        image_path (str): The path to the image file.

    Returns:
        torch.Tensor: The PyTorch tensor representing the image.
    """
    image = cv2.imread(image_path)
    array = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    tensor = convert_to_tensor(array)
    return tensor

def load_image_pytorch(image_path):
    tensor = read_image(image_path)
    return tensor

In [11]:
from rich.console import Console
from rich.table import Table
console = Console()

def create_table_rich(num_pixels):
    """
    Creates a rich table object with specific columns for displaying statistics of an image.

    Args:
        num_pixels (int): The number of pixels in the image.

    Returns:
        rich.table.Table: The rich table object.
    """
    # Create a new table with a title that includes the number of pixels
    table = Table(title=f"Statistics Table for image with num pixels = {num_pixels}")

    # Add columns to the table with specified styles
    table.add_column("Name", justify="right", style="cyan", no_wrap=True)
    table.add_column("First iteration (ms)", justify="right", style="cyan", no_wrap=True)
    table.add_column("Mean (ms)", style="magenta")
    table.add_column("median (ms)", justify="right", style="green")
    table.add_column("Throughput (Megapixel / sec)", justify="right", style="green")
    table.add_column("fps (fps)", justify="right", style="green")

    return table

def benchmark_image_loading(methods,basesetup, sizes, image_format="jpg"):
    """
    Benchmark the loading of images using different methods and different image sizes.

    Args:
        methods (dict): A dictionary containing the methods to benchmark.
        basesetup (str): Base setup string for the benchmarking environment.
        sizes (list): A list of image sizes to benchmark.

    Returns:
        pandas.DataFrame: The benchmark results in a DataFrame format.
    """
    console.print("Creating and saving dummy images to disk :")
    # Creating and saving dummy images to disk
    for size in sizes :
        array = np.random.randint(0, 255, size=(size, size, 3), dtype=np.uint8)
        if image_format == "jpg":
            pil_image = Image.fromarray(array)
            pil_image.save(f"image_{size}.{image_format}")
        elif image_format=="tif":
            tiff.imwrite(f"image_{size}.{image_format}", array, bigtiff=True, photometric='rgb')
        
    # Empty results
    results = []

    for i in sizes:
        image_path =f"image_{i}.{image_format}"
        img = Image.open(image_path)
        height, width = img.size

        num_pixels = height * width
        num_runs = 10

        console.rule(f"Benchmark statistics : Image with shape = {img.size} and {image_format} format | N° Runs = {num_runs}  ")

        globals1 = {'image_path':image_path}

        table = create_table_rich(num_pixels)

        for method_name, method in methods.items():

            try : 

                start = time.time()
                torch.cuda.synchronize() 
                method(image_path).to(device='cuda:0')
                torch.cuda.synchronize() 
                first_load = (time.time()-start)*1000
                stmt1 = f"{method_name}(image_path).to(device='cuda:0')"
                setup = f"{basesetup}\n{inspect.getsource(method)}\n{inspect.getsource(pil_to_numpy)}\nconvert_to_tensor = T.transforms.ToTensor()"

                t = Timer(stmt=stmt1, setup=setup, globals=globals1).timeit(num_runs)

                name, mean, median, throughput, fps = method_name, t.mean * 1000, t.median * 1000, num_pixels / 1e6 / t.median, 1 / t.median
                results.append({
                    "name":name,
                    "first_load":first_load,
                    "mean":mean,
                    "meadian":median,
                    "throughput":throughput,
                    "fps":fps,
                    "num_pixels":num_pixels,
                })
                table.add_row(name,f"{first_load:.3f}" , f"{mean:.3f}",  f"{median:.3f}", f"{throughput:.3f}", f"{fps:.3f}" )
            except RuntimeError as e :
                print(f"{method_name} incompatible with {image_format}")
                continue

        
        console.print(table)
    df = pd.DataFrame(results)
    return df 

# disk read time

In [10]:
Image.fromarray(np.ones((10000, 10000, 3), np.uint8)).save("dummy.jpg")
!time dd if=dummy.jpg of=/dev/null bs=1M count=1000

1+1 records in
1+1 records out
1563127 bytes (1.6 MB, 1.5 MiB) copied, 0.00160811 s, 972 MB/s

real	0m0.004s
user	0m0.004s
sys	0m0.001s


# Run exp

In [12]:
methods = {
    "load_image_nvjpegl_cpu": load_image_nvjpegl_cpu,
    "load_image_nvjpegl_gpu": load_image_nvjpegl_gpu,
    "load_image_skimage": load_image_skimage,
    "load_image_pil": load_image_pil,
    "load_image_pil_accelerated": load_image_pil_accelerated,
    "load_image_cv2": load_image_cv2,
    "load_image_rasterio":load_image_rasterio,
    "load_image_pil_to_tensor": load_image_pil_to_tensor,
    "load_image_cv_to_tensor": load_image_cv_to_tensor,
    "load_image_pytorch": load_image_pytorch,
}
setups = {
    "load_image_nvjpegl_cpu": "from torchvision.io.image import decode_jpeg, read_file",
    "load_image_nvjpegl_gpu": "from torchvision.io.image import decode_jpeg, read_file",
    "load_image_skimage": "from skimage import io",
    "load_image_pil": "from PIL import Image",
    "load_image_pil_accelerated": "from PIL import Image\nimport numpy as np",
    "load_image_cv2": "import cv2",
    "load_image_rasterio":"import rasterio",
    "load_image_pil_to_tensor": "from PIL import Image\nfrom torchvision import transforms as T",
    "load_image_cv_to_tensor": "import cv2\nfrom torchvision import transforms as T",
    "load_image_pytorch": "from torchvision.io import read_image"
}
basesetup = """import torch\nfrom torch.utils.benchmark import Timer\n"""
basesetup += "\n".join(list(setups.values()))

sizes =(100,200, 300, 500, 1000, 2000,5000, 10000)
image_format = "jpg"
df = benchmark_image_loading(methods,basesetup,sizes, image_format)

Creating and saving dummy images to disk :

────────────── Benchmark statistics : Image with shape = (100, 100) and jpg format | N° Runs = 10   ───────────────

/home/tommy-local/projects/smartem/.venv/lib/python3.10/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


                                Statistics Table for image with num pixels = 10000                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃                            ┃                      ┃           ┃             ┃ Throughput (Megapixel ┃           ┃
┃                       Name ┃ First iteration (ms) ┃ Mean (ms) ┃ median (ms) ┃                / sec) ┃ fps (fps) ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│     load_image_nvjpegl_cpu │               14.597 │ 0.182     │       0.182 │                54.840 │  5483.992 │
│     load_image_nvjpegl_gpu │                3.268 │ 1.490     │       1.490 │                 6.711 │   671.076 │
│         load_image_skimage │                1.776 │ 0.627     │       0.627 │                15.941 │  1594.078 │
│             load_image_pil │                0.544 │ 0.307     │       0.307 │                32.593 │  3259.314 │
│ load_image_pil_accelerated │                0.545 │ 0.296     │       0.296 │                33.755 │  3375.460 │
│             load_image_cv2 │                0.548 │ 0.214     │       0.214 │                46.632 │  4663.152 │
│        load_image_rasterio │                3.009 │ 1.148     │       1.148 │                 8.709 │   870.891 │
│   load_image_pil_to_tensor │                0.784 │ 0.455     │       0.455 │                22.002 │  2200.170 │
│    load_image_cv_to_tensor │                0.581 │ 0.362     │       0.362 │                27.651 │  2765.107 │
│         load_image_pytorch │                0.532 │ 0.206     │       0.206 │                48.438 │  4843.757 │
└────────────────────────────┴──────────────────────┴───────────┴─────────────┴───────────────────────┴───────────┘

────────────── Benchmark statistics : Image with shape = (200, 200) and jpg format | N° Runs = 10   ───────────────

...

───────────── Benchmark statistics : Image with shape = (5000, 5000) and jpg format | N° Runs = 10   ──────────────

                               Statistics Table for image with num pixels = 25000000                               
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃                            ┃                      ┃           ┃             ┃ Throughput (Megapixel ┃           ┃
┃                       Name ┃ First iteration (ms) ┃ Mean (ms) ┃ median (ms) ┃                / sec) ┃ fps (fps) ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│     load_image_nvjpegl_cpu │              269.714 │ 249.509   │     249.509 │               100.197 │     4.008 │
│     load_image_nvjpegl_gpu │              210.515 │ 206.644   │     206.644 │               120.981 │     4.839 │
│         load_image_skimage │              380.767 │ 349.560   │     349.560 │                71.518 │     2.861 │
│             load_image_pil │              345.961 │ 348.562   │     348.562 │                71.723 │     2.869 │
│ load_image_pil_accelerated │              274.759 │ 275.457   │     275.457 │                90.758 │     3.630 │
│             load_image_cv2 │              398.271 │ 397.105   │     397.105 │                62.956 │     2.518 │
│        load_image_rasterio │              853.670 │ 852.023   │     852.023 │                29.342 │     1.174 │
│   load_image_pil_to_tensor │              514.097 │ 744.171   │     744.171 │                33.594 │     1.344 │
│    load_image_cv_to_tensor │              540.920 │ 797.154   │     797.154 │                31.362 │     1.254 │
│         load_image_pytorch │              229.586 │ 248.772   │     248.772 │               100.494 │     4.020 │
└────────────────────────────┴──────────────────────┴───────────┴─────────────┴───────────────────────┴───────────┘

/home/tommy-local/projects/smartem/.venv/lib/python3.10/site-packages/PIL/Image.py:3074: DecompressionBombWarning: Image size (100000000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


──────────── Benchmark statistics : Image with shape = (10000, 10000) and jpg format | N° Runs = 10   ─────────────

KeyboardInterrupt: 

In [13]:
image_format = "tif"
df = benchmark_image_loading(methods,basesetup,sizes, image_format)

Creating and saving dummy images to disk :

────────────── Benchmark statistics : Image with shape = (100, 100) and tif format | N° Runs = 10   ───────────────

load_image_nvjpegl_cpu incompatible with tif
load_image_nvjpegl_gpu incompatible with tif
load_image_pytorch incompatible with tif


                                Statistics Table for image with num pixels = 10000                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃                            ┃                      ┃           ┃             ┃ Throughput (Megapixel ┃           ┃
┃                       Name ┃ First iteration (ms) ┃ Mean (ms) ┃ median (ms) ┃                / sec) ┃ fps (fps) ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│         load_image_skimage │                1.171 │ 0.404     │       0.404 │                24.769 │  2476.864 │
│             load_image_pil │                0.878 │ 0.461     │       0.461 │                21.691 │  2169.114 │
│ load_image_pil_accelerated │                0.729 │ 0.423     │       0.423 │                23.631 │  2363.085 │
│             load_image_cv2 │                2.044 │ 0.109     │       0.109 │                91.854 │  9185.351 │
│        load_image_rasterio │                2.814 │ 1.289     │       1.289 │                 7.755 │   775.502 │
│   load_image_pil_to_tensor │                1.173 │ 0.641     │       0.641 │                15.610 │  1560.972 │
│    load_image_cv_to_tensor │                0.431 │ 0.215     │       0.215 │                46.511 │  4651.140 │
└────────────────────────────┴──────────────────────┴───────────┴─────────────┴───────────────────────┴───────────┘

────────────── Benchmark statistics : Image with shape = (200, 200) and tif format | N° Runs = 10   ───────────────

...

load_image_nvjpegl_cpu incompatible with tif
load_image_nvjpegl_gpu incompatible with tif
load_image_pytorch incompatible with tif


                               Statistics Table for image with num pixels = 25000000                               
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃                            ┃                      ┃           ┃             ┃ Throughput (Megapixel ┃           ┃
┃                       Name ┃ First iteration (ms) ┃ Mean (ms) ┃ median (ms) ┃                / sec) ┃ fps (fps) ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│         load_image_skimage │               53.495 │ 52.511    │      52.511 │               476.088 │    19.044 │
│             load_image_pil │              134.679 │ 131.148   │     131.148 │               190.625 │     7.625 │
│ load_image_pil_accelerated │              111.583 │ 110.136   │     110.136 │               226.991 │     9.080 │
│             load_image_cv2 │              194.853 │ 112.302   │     112.302 │               222.614 │     8.905 │
│        load_image_rasterio │               82.367 │ 68.178    │      68.178 │               366.686 │    14.667 │
│   load_image_pil_to_tensor │              297.467 │ 509.360   │     509.360 │                49.081 │     1.963 │
│    load_image_cv_to_tensor │              242.972 │ 486.521   │     486.521 │                51.385 │     2.055 │
└────────────────────────────┴──────────────────────┴───────────┴─────────────┴───────────────────────┴───────────┘

──────────── Benchmark statistics : Image with shape = (10000, 10000) and tif format | N° Runs = 10   ─────────────

load_image_nvjpegl_cpu incompatible with tif
load_image_nvjpegl_gpu incompatible with tif
load_image_pytorch incompatible with tif


                              Statistics Table for image with num pixels = 100000000                               
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃                            ┃                      ┃           ┃             ┃ Throughput (Megapixel ┃           ┃
┃                       Name ┃ First iteration (ms) ┃ Mean (ms) ┃ median (ms) ┃                / sec) ┃ fps (fps) ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│         load_image_skimage │              392.418 │ 208.412   │     208.412 │               479.818 │     4.798 │
│             load_image_pil │              872.690 │ 643.321   │     643.321 │               155.443 │     1.554 │
│ load_image_pil_accelerated │              545.869 │ 455.618   │     455.618 │               219.482 │     2.195 │
│             load_image_cv2 │             1018.869 │ 583.890   │     583.890 │               171.265 │     1.713 │
│        load_image_rasterio │              274.610 │ 237.452   │     237.452 │               421.137 │     4.211 │
│   load_image_pil_to_tensor │             1014.686 │ 2178.634  │    2178.634 │                45.900 │     0.459 │
│    load_image_cv_to_tensor │             1073.883 │ 2177.260  │    2177.260 │                45.929 │     0.459 │
└────────────────────────────┴──────────────────────┴───────────┴─────────────┴───────────────────────┴───────────┘